In [1]:
#imports

# basic data handling
import numpy as np
import pandas as pd

# raster handling
import rasterio as ras
from rasterio import plot
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.enums import Resampling as resa
import affine
from rasterio.warp import reproject
from matplotlib import pyplot as plt

# API
from sentinelsat.sentinel import SentinelAPI

# vector handling
import geopandas as gpd
from osgeo import gdal
import pyproj
import folium as fol

# file/sys handling
import os
import glob
import shutil
from zipfile import ZipFile

#import raster_functions
exec(open('raster_functions.py').read())

### Check the field boundary list and tile

In [2]:
### user input ###
proj_fold = 'C:\\Users\\User\\geospatial_projects\\s2arch'
image_fold = 'C:\\Users\\User\\geospatial_projects\\s2arch\\s2_SR'
###

# list Geojsons
boundary_set = glob.glob(proj_fold+'\\'+'crop_bounds'+'\\'+'*.geojson')

# drawing boundaries
print('Drawing ROI')
print('-------------------')
# create a single temporary geojson
gdf_list = []
for bound in boundary_set:
    print(bound[53:-8])
    bound_gdf = gpd.read_file(bound)
    # append empty list with geodfs
    gdf_list.append(bound_gdf)

# concatanate gdf
combo_gdf = gpd.GeoDataFrame(pd.concat(gdf_list))[['Name','geometry']]    
#save  acopy 
combo_gdf.to_file('combo_shapes.geojson', driver='GeoJSON')

# drawing
map_set = fol.Map(location=[6.7, 80.7], zoom_start=10, tiles='openstreetmap')
fol.GeoJson(data=open('combo_shapes.geojson', 'r', encoding='utf-8-sig').read()).add_to(map_set)
#adding s2 tile
fol.GeoJson(data=open('tile_footprint.geojson', 'r', encoding='utf-8-sig').read()).add_to(map_set)
map_set

Drawing ROI
-------------------
estate_1_pedro_NE
estate_2_mahauva_NE
estate_3_Bramly_NE
estate_4_thalawakale_NE
estate_5_queenwood_NW
paddy_0_meegaswewa_HB
sugar_0_udawalawa_AMB
sugar_1_kowulara_AMB
sugar_2_udawalawa_AMB


### check products

In [3]:
#### user input ####
start_date = '2020-01-01'
end_date = '2020-02-01'

# cloud cover
cc = 80

# sentinel credentials
user= 'ymaramb'
password = 'Deb!1986anur'
######################

# foot print
footprint = None
for i in combo_gdf['geometry']:
    footprint = i


# download month to month to avoid memory jams
# satrt days
start_D = pd.date_range(start=start_date, end=end_date, freq='M')-pd.offsets.MonthBegin(1)
start_Days = list(pd.Series(start_D.strftime('%Y%m%d')))
# end days
end_D = pd.date_range(start=start_date, end=end_date, freq='M')
end_Days = list(pd.Series(end_D.strftime('%Y%m%d')))

# loop over days for loading scenes
for s_day, e_day in zip(start_Days, end_Days):
    print('Downloading for-' + s_day, e_day)
    # api call
    api = SentinelAPI(user, password,
                  'https://scihub.copernicus.eu/dhus')
    # query
    products = api.query(footprint,
                    date = (s_day, e_day),
                    platformname='Sentinel-2',
                    processinglevel = 'Level-2A',
                    cloudcoverpercentage = (0,cc))
    # extractprod info to a gdf
    product_gdf = api.to_geodataframe(products)
    gdf_sorted = product_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
    s2_gdf = gdf_sorted[['title','cloudcoverpercentage', 'processinglevel', 'uuid']]
    # uuid listing
    s2_list = list(s2_gdf.uuid)
    
    # download for uuids
    for uuid in s2_list:
        print(uuid)
        api.download(uuid)
        
###### from here script can be break only for downloading images ############
###### and only for tif/png extraction ######################################

        # extract data from zip
        zips = glob.glob("*.zip")
        # zip file path obj
        zf = ZipFile(zips[0])
        # extract zip
        zf.extractall(proj_fold+'\\'+'s2_raw')
        zf.close()
        # delete zip
        os.remove(zips[0])
        
        # safe files in the dir
        safe = glob.glob(proj_fold+'\\'+'s2_raw'+'\\*SAFE')
        
        # find folder containig granule
        for root, dirs, files in os.walk(safe[0]):
            for fols in dirs:
                if fols.startswith('L2A'):
                    su_fol = fols
                    print(su_fol)
         
        # date and tile strings for band band name
        time_str = safe[0][-54:-39]
        tile_str = safe[0][-27:-21]
        band_str = tile_str+'_'+time_str+'_'
        band_path = safe[0]+'\\GRANULE\\'+su_fol+'\\IMG_DATA\\'
        
        # open bands for restacking
        blue = ras.open(safe[0]+'\\GRANULE\\'+su_fol+'\\IMG_DATA\\R10m\\'+band_str+'B02_10m.jp2')
        green = ras.open(safe[0]+'\\GRANULE\\'+su_fol+'\\IMG_DATA\\R10m\\'+band_str+'B03_10m.jp2')
        red = ras.open(safe[0]+'\\GRANULE\\'+su_fol+'\\IMG_DATA\\R10m\\'+band_str+'B04_10m.jp2')
        nir = ras.open(safe[0]+'\\GRANULE\\'+su_fol+'\\IMG_DATA\\R10m\\'+band_str+'B08_10m.jp2')
        
        # open band 8a and crete a 10m array
        # from the custom function 10m resolution nir 8a array
        nira_res =  raster_resample(safe[0]+'\\GRANULE\\'+su_fol+'\\IMG_DATA\\R20m\\'+band_str+'B8A_20m.jp2', 2)
        # meta data for saving image
        meta_8a = nir.profile
        
        # write the 8a band to jp2 again
        with ras.open(safe[0]+'\\GRANULE\\'+su_fol+'\\IMG_DATA\\R20m\\'+band_str+'B8A_10m.jp2', 'w', **meta_8a) as res_dst:
            res_dst.write(nira_res)
        # open nira band for stacking
        nira = ras.open(safe[0]+'\\GRANULE\\'+su_fol+'\\IMG_DATA\\R20m\\'+band_str+'B8A_10m.jp2')
        
        with ras.open('image_name.tiff','w',driver='Gtiff', width=blue.width, height=blue.height, count=5, crs=blue.crs,transform=blue.transform, dtype=blue.dtypes[0]) as composite:
            # write each band to composite
            composite.write(blue.read(1),1)
            composite.write(green.read(1),2)
            composite.write(red.read(1),3)
            composite.write(nir.read(1),4)
            composite.write(nira.read(1),5)
            # close the opend data
            composite.close()
        
        #close all opend data sets
        blue.close()
        green.close()
        red.close()
        nir.close()
        nira.close()
        
        #Delete SAFE and ZIP
        try:
            shutil.rmtree(safe[0])
        except:
            pass
        
        # Delete zip file
        zf.close()
        try:
            os.remove(zips[0])
        except:
            pass
        
        image_folder = None
        # extract crs from image
        check  = ras.open('image_name.tif')
        # reformatting
        epsg_str = 'epsg:'+str(check.crs)[-5:]
        
        # clipping the tile for rio tifs
        # iterate over boundaries
        for bound in boundary_set:
            print(bound[53:-8])
            # gdf
            bound_gdf = gpd.read_file(bound)
            # crs from image
            bound_crs = bound_gdf.to_crs({'init':epsg_str})
            # extract tittle with roi name and date from image
            im_title = bound[53:-8]+su_fol[18:27]
            # saving folder name
            image_folder = image_fold+'\\'+bound[53:-8]
            
            # create folder for saving
            try:
                os.mkdir(image_folder)
            except:
                pass
            
            # masking image tile with roi shape
            with ras.open('image_name.tif') as src:
                out_image, out_transform = mask(src, bound_crs.geometry, crop=True)
                out_meta = src.meta.copy()
                out_meta.update({'driver':'Gtiff',
                                 'height':out_image.shape[1],
                                 'width':out_image.shape[2],
                                 'transform':out_transform})
                
            # writing image for roi
            with ras.open(image_folder+'\\'+im_title+'.tif', 'w', **out_meta) as final:
                final.write(out_image)
            
            #close src data set
            src.close()
            #open data for plotting
            src_fin = ras.open(image_folder+'\\'+im_title+'.tif')
            # read into arrays
            blue = src_fin.read(1)
            green = src_fin.read(2)
            red = src_fin.read(3)
            nir = src_fin.read(5)
    
            #ndvi = (nir-red)/(nir+red)
    
            ax, fig = plt.subplots(figsize=(10,8))
            #plt.imshow(ndvi, cmap='RdYlGn')
            #clb = (plt.colorbar(orientation='vertical', shrink=0.75, fraction=0.2))
    
    
            #normalized bands
            redn = normalize(red)
            greenn = normalize(green)
            bluen = normalize(blue)
    
    
            #print(" red normalized   max - {val1} ::: min - {val2}".format(val1=redn.max(),val2= redn.min()))
            #print(" green normalized max - {val1} ::: min - {val2}".format(val1=greenn.max(),val2= greenn.min()))
            #print(" blue normalized  max - {val1} ::: min - {val2}".format(val1=bluen.max(),val2= bluen.min()))
    
            # creating alpha band from scrach
            alpha = create_alpha_channel(redn, bluen) 
            # stacking arrays (create RGB composite) and alpha stack(to crate RGBA)
            rgb = np.dstack((redn, greenn, bluen, alpha))
            #rgb = np.dstack((redn, greenn, bluen))

            # vizualize image
            plt.imshow(rgb)
            png_path = image_folder+'\\'+im_title+'.png'
            plt.savefig(png_path)
            plt.show()
            plt.close()

C:\Users\User\miniconda3\envs\geospatial\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


bc5f6d74-a04a-4fa7-bb44-094a72c55848


Downloading: 100%|██████████████████████████| 828M/828M [03:29<00:00, 3.96MB/s]
MD5 checksumming: 100%|██████████████████████| 828M/828M [00:03<00:00, 217MB/s]


L2A_T44NMN_A023668_20200103T050602


RasterioIOError: image_name.tif: No such file or directory